PREPROCESSING

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import datetime

In [4]:
raw_data = pd.read_csv('../csv/caspecoTrainingData.csv', header=0)
holiday_data = pd.read_csv('../csv/swedish_holidays.csv', header=0)
raw_data = raw_data.sort_values('Date').reset_index(drop=True)
raw_data.head()
holiday_data.head()

,Date,Day
0,2020-01-01,New Year Day
1,2020-01-06,Epiphany
2,2020-04-10,Good Friday
3,2020-04-12,Easter Day
4,2020-04-13,Easter Monday


In [16]:
def missing_dates(df):
    date_counts = df["Date"].value_counts()[df["Date"].sort_values()].to_dict()
    t = np.datetime_as_string(np.arange(datetime.datetime(2020,1,1), datetime.datetime(2023,1,4), datetime.timedelta(days=1)),unit="D")
    missing_dates = []
    for date in t:
        if date not in date_counts :
            missing_dates.append(date)
        if date_counts[date] < 3:
            missing_dates.append(date)
    return missing_dates

def add_dates(lst, df):
    
    comp = np.sort(df['Company'].unique())

    for date in lst:

        new_df = df[df['Date'] == date]

        for c in comp:

            if c not in new_df['Company'].values:  
                new_row = pd.DataFrame({"Date": [date], "Company": [c],"Sales": [-1]})
                df = pd.concat([df, new_row], ignore_index=True)
    
    df = df.sort_values('Date').reset_index(drop=True)
    return df

def add_holidays(df, df_holiday):

    if 'is_holiday' in df.columns:
        return df
    
    df['is_holiday'] = 0

    for date in df_holiday['Date']:
        df.loc[df['Date'] == date,'is_holiday'] = 1

    return df

def add_weekdays(df):

    if 'weekday' in df.columns:
        return df
    
    df['weekday'] = 0

    for i, date in enumerate(df['Date']):

        df.at[i, 'weekday'] = datetime.datetime.strptime(date, "%Y-%m-%d").weekday()

    return df


missing = missing_dates(raw_data)
df_dates = add_dates(missing, raw_data)
df_holiday = add_holidays(df_dates, holiday_data)
df_days = add_weekdays(df_holiday)

df_days.head()

,Date,Company,Sales,is_holiday,weekday
0,2020-01-01,2,72608.623649,1,2
1,2020-01-01,1,-1.000000,1,2
2,2020-01-01,0,-1.000000,1,2
3,2020-01-02,1,57957.790779,0,3
4,2020-01-02,0,25421.830339,0,3
